# Data visualization
### The Computational tool for generating a porous scaffold based on implicit surfaces


**Abstract**

This notebook visualized the results from the publication. The raw dataset is available at [Mendeley Data](https://data.mendeley.com/datasets/sbxr7xxvnd/2).
The program that used to generate this data released at [Github repository](https://github.com/nodtem66/Scaffolder). In this notebook, you will explore the interactive results from the frequency angle and iso-level study. The regression analysis also possible to re-evaluate. 

<br>
Jirawat Iamsamang<sup>1</sup>, Phornphop Naiyanetr<sup>1,*</sup><br>
<sup>1</sup>Dept. of Biomedical Engineering, Faculty of Engineering,
Mahidol University, Nakhon Pathom, Thailand
<sup>*</sup>Corresponding author: phornphop.nai@mahidol.ac.th




# Frequency angle and iso-level study

In [1]:
#@title  Download dataset from Mendeley
! curl -L https://data.mendeley.com/public-files/datasets/sbxr7xxvnd/files/3a586aee-6d38-48b3-9d75-f4b6eeb42b6d/file_downloaded -o freq_iso_level.xlsx
! pip install -U plotly

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   123  100   123    0     0    129      0 --:--:-- --:--:-- --:--:--   129
100  141k  100  141k    0     0  80678      0  0:00:01  0:00:01 --:--:-- 23.1M
     |████████████████████████████████| 12.9MB 6.9MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [258]:
#@title Helper functions
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rc
from matplotlib.ticker import MultipleLocator
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import json
import os

rc('font',size=8)
rc('font',family='serif')
rc('axes',labelsize=10)

def plot(name, x, y, z, is_export=True):
    fig = plt.figure(name)
    # check that y data have more than one sample
    if pd.unique(y).shape[0] <= 1:
        # plot 2D
        sns.scatterplot(x, z)
        plt.xlabel('$\\omega$')
        plt.show()
    else: 
        # plot 3D
        ax = fig.gca(projection='3d')
        ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.2)
        # Backgroud
        ax.grid(True)
        ax.xaxis.pane.set_edgecolor('black')
        ax.yaxis.pane.set_edgecolor('black')
        ax.xaxis.pane.fill = False
        ax.yaxis.pane.fill = False
        ax.zaxis.pane.fill = False
        ax.xaxis._axinfo['tick']['inward_factor'] = 0
        ax.xaxis._axinfo['tick']['outward_factor'] = 0.4
        ax.yaxis._axinfo['tick']['inward_factor'] = 0
        ax.yaxis._axinfo['tick']['outward_factor'] = 0.4
        ax.zaxis._axinfo['tick']['inward_factor'] = 0
        ax.zaxis._axinfo['tick']['outward_factor'] = 0.4
        ax.zaxis._axinfo['tick']['outward_factor'] = 0.4

        plt.xlabel('$\\omega$')
        plt.ylabel('t')
        plt.show()

def visualize(data, surface='bcc', is_export=True):
    df = data.query(f'Surface == "{surface}"')
    plot(f'{surface}--min', df.w, df.t, df.Min_Feret, is_export)
    plot(f'{surface}--max', df.w, df.t, df.Max_Feret, is_export)
    plot(f'{surface}--porosity', df.w, df.t, df.Porosity, is_export)
    plot(f'{surface}--surface', df.w, df.t, df.SAR, is_export)

def regress(data, surface='bcc', formula=''):
    df = data.query(f'Surface == "{surface}"')
    try:
      model = smf.ols(formula=formula, data=df)
      res = model.fit()
      print(res.summary())
    except Exception as ex:
      print(ex)

def show_corr(data):
    corr = data.corr()
    fig, _ = plt.subplots()
    corr_columns = ['$\\omega$', '$t$', '$\\phi_{min}$', '$\\phi_{max}$', 'P', 'SAR']
    sns.heatmap(corr, xticklabels=corr_columns, yticklabels=corr_columns, cmap=sns.color_palette("RdBu", n_colors=21), center=0, vmin=-1, vmax=1)
    plt.show()

def p(x, d=2):
    return np.power(x,d)

def r(x, d=1, eps=0):
    return np.reciprocal(x + eps) if d is 1 else np.power(np.reciprocal(x + eps), d)

In [4]:
#@title Prepare data
# Read data into DataFrame
data = pd.read_excel('freq_iso_level.xlsx', sheet_name='Sheet1')
data.columns = ['Surface', 'w', 't', 'Min_Feret', 'Max_Feret', 'Porosity', 'SAR']
# Remove the empty data
invalid = data.query('Porosity == 1')
data.drop(invalid.index, inplace=True)
invalid = data.query('Min_Feret == 0')
data.drop(invalid.index, inplace=True)

# Remove the missing data
# Criterion: the number of records with the same w and surface < 5
countData = data.groupby(['Surface', 'w']).count()
for k,v in (countData.t < 5).items():
    if v and k[0] not in ['tubular_g_c', 'tubular_g_ab']:
        data.drop(data.query(f'Surface == \'{k[0]}\' and w == {k[1]}').index, inplace=True)
# Criterion: the number of records with the same t and surface < 5
countData = data.groupby(['Surface', 't']).count()
for k,v in (countData.w < 5).items():
    if v and k[0] not in ['tubular_g_c', 'tubular_g_ab']:
        data.drop(data.query(f'Surface == \'{k[0]}\' and t == {k[1]}').index, inplace=True)

In [103]:
#@title Correlation heat map
corr = data.corr()
corr_columns = ['$\\omega$', '$t$', '$\\phi_{min}$', '$\\phi_{max}$', 'P', 'SAR']
fig = go.Figure(data=go.Heatmap(
                   z=corr[::-1],
                   x=corr_columns,
                   y=corr_columns[::-1],
                   colorscale="RdBu",
                   hoverongaps = False))
fig.update_xaxes(side="top")
fig.show()

In [289]:
#@title Plot surface helper function
@interact(surface=data.Surface.unique())
def plot_surface(surface):
  
  if surface in ['tubular_g_ab', 'tubular_g_c']:
    visualize(data, surface, False)
    return

  df = data.query(f'Surface == "{surface}"')
  x = df.w.unique()
  y = df.t.unique()
  df.set_index(['w', 't'], inplace=True)
  df = df[~df.index.duplicated(keep='first')]
  
  def fill(x):
    missing = [i for i, x in enumerate(x) if x is None]
    if len(missing) == len(x):
      return []
    for k,l in enumerate(missing):
      if k is not l:
        x[l] = x[l-1]
    missing = [i for i, x in enumerate(x) if x is None]
    if len(missing) > 0:
      f = x[missing[-1]+1]
      for k in missing:
        x[k] = f
    return x

  def get_surface(column_name, colorscale=None):
    z = []
    for i in y:
      _z = []
      for j in x:
        try:
          _z.append(df.loc[(j,i), column_name])
        except:
          _z.append(None)
      _z = fill(_z)
      if len(_z) > 0:
        z.append(_z)
    z = np.array(z, dtype='float32')
    return go.Surface(
      z=z, x=x, y=y,
      colorscale=colorscale,
      showscale=False
    )
  
  fig = make_subplots(
    rows=2, cols=2,
    horizontal_spacing=.1, vertical_spacing=.1,
    specs=[[{'type':'surface'},{'type':'surface'}], [{'type':'surface'},{'type':'surface'}]],
    subplot_titles=("Min_Feret", "Max_Feret", "Porosity", "SAR")
  )
  fig.add_trace(get_surface('Min_Feret'), row=1, col=1)
  fig.add_trace(get_surface('Max_Feret'), row=1, col=2)
  fig.add_trace(get_surface('Porosity'), row=2, col=1)
  fig.add_trace(get_surface('SAR'), row=2, col=2)

  
  #fig = go.Figure(data=[get_surface('SAR')])
  fig.update_layout(title=surface,
    width=800, height=800,
    scene1=dict(aspectmode='cube', camera=dict(eye={'x':1.5, 'y':-1.5, 'z':1.5})),
    scene2=dict(aspectmode='cube', camera=dict(eye={'x':1.5, 'y':-1.5, 'z':1.5})),
    scene3=dict(aspectmode='cube', camera=dict(eye={'x':1.5, 'y':-1.5, 'z':1.5})),
    scene4=dict(aspectmode='cube', camera=dict(eye={'x':-1.5, 'y':1.5, 'z':1.5})),
    #margin=dict(l=5, r=0, b=5, t=0)
  )
  fig.show()

interactive(children=(Dropdown(description='surface', options=('bcc', 'double-d', 'double-gyroid', 'double-p',…

In [260]:
#@title Regression analysis
#@markdown **Symbol**: w, t, Min_Feret, Max_Feret, Porosity, SAR<br>
#@markdown **Syntax**: [Statsmodels R-style](https://www.statsmodels.org/devel/example_formulas.html)<br>
#@markdown **Example:**
#@markdown
#@markdown * `Min_Feret ~ np.reciprocal(w)*t - 1` 
#@markdown * `Max_Feret ~ r(w)*t - 1` 
#@markdown * `Porosity ~ w*p(t)`
#@markdown * `SAR ~ w*np.power(t,2)`
#@markdown
#@markdown PS. `r(x) := np.reciprocal(x)` and `p(x, d=2) := np.power(x,d)`
interact(regress, data=fixed(data), surface=data.Surface.unique(), formula='')

interactive(children=(Dropdown(description='surface', options=('bcc', 'double-d', 'double-gyroid', 'double-p',…

<function __main__.regress>